# Tarea 2, pregunta 1: "RNN sobre texto"

## Integrantes

* _**Francisco Olivares - 201473575-8 - fracisco.olivars.14@sansano.usm.cl**_
* _**Felipe Vega - 201473511-1 - felipe.vega.14@sansano.usm.cl**_

In [2]:
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow GPU stuff :v
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

# Modificar estilos de jupyter
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.text_cell_render p, .text_cell_render li { font-size: 12.5pt !important; }</style>"))
display(HTML("<style>.text_cell_render li { margin: 0 0 10px 0; }</style>"))

## a)

In [4]:
df_ner = pd.read_csv("./dataset_1/ner.csv", encoding ="cp1252", error_bad_lines=False)
df_ner.dropna(inplace=True)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [12]:
dataset = df_ner.loc[:,["lemma","word","pos","tag","prev-iob"]]

# Cantidad de lapabras a utilizar
n_used = 500000
dataX_raw, dataY_raw = [],[]
lemmas, labels = set(), set()  #uniques

initial_sentence = True
sentence= []
labels_sentence = []

for fila in dataset.values[:n_used]:
    if fila[-1]=="__START1__":
        
        
        if initial_sentence:
            initial_sentence = False
            
        else:
            dataX_raw.append(sentence)
            dataY_raw.append(labels_sentence)
            sentence= []
            labels_sentence = []
        
    lemmas.add(fila[0])
    labels.add(fila[3])
    
    sentence.append(fila[0]) #add lemma
    labels_sentence.append(fila[3]) #TAG
    
# dataX_raw = dataX_raw[1:]
# dataY_raw = dataY_raw[1:]

In [21]:
print("Cantidad de frases:", len(dataX_raw))
print("Cantidad de tags:", len(dataY_raw))


len_of_sentence = set([len(i) for i in dataX_raw])
len_of_tags = set([len(i) for i in dataY_raw])

print("Sentencia más corta: {}, más larga: {}, largos distintos: {}".format(min(len_of_sentence), max(len_of_sentence), len(len_of_sentence)))
print("Sentencia más corta: {}, más larga: {}, largos distintos: {}".format(min(len_of_tags), max(len_of_tags), len(len_of_tags)))

Cantidad de frases: 22901
Cantidad de tags: 22901
Sentencia más corta: 1, más larga: 70, largos distintos: 62
Sentencia más corta: 1, más larga: 70, largos distintos: 62
